In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score,confusion_matrix
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, LSTM, Dense, TimeDistributed, Dropout
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import gc
import tensorflow as tf
import seaborn as sns

In [ ]:
def preprocess_frame(frame):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (112, 112))
    return frame / 255.0

In [ ]:
# Function for face detection
def detect_face(frame):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    if len(faces) > 0:
        (x, y, w, h) = faces[0]
        return frame[y:y+h, x:x+w]
    return None


In [ ]:
# Load video and extract frames
def extract_frames(video_path, num_frames=10):
    cap = cv2.VideoCapture(video_path)
    frames = []
    frame_count = 0
    while len(frames) < num_frames and cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if frame_count % 3 == 0:  # Sample every 3rd frame
            face = detect_face(frame)
            if face is not None:
                face = preprocess_frame(face)
                frames.append(face)
        frame_count += 1
    cap.release()

    # If we couldn't extract enough frames, duplicate the last frame
    while len(frames) < num_frames:
        frames.append(frames[-1] if frames else np.zeros((112, 112, 3)))

    return np.array(frames[:num_frames])


In [ ]:
# Load dataset
def load_dataset(real_dir, fake_dir,max_videos=100):
    X = []
    y = []

    # Load real videos
    for video_name in os.listdir(real_dir):
        video_path = os.path.join(real_dir, video_name)
        frames = extract_frames(video_path)
        X.append(frames)
        y.append(0)  # 0 for real

    # Load fake videos
    for video_name in os.listdir(fake_dir):
        video_path = os.path.join(fake_dir, video_name)
        frames = extract_frames(video_path)
        X.append(frames)
        y.append(1)  # 1 for fake

    return np.array(X), np.array(y)

In [ ]:
# Build the model
def build_model(input_shape):
    model = Sequential()

    # Feature extraction using ResNet50
    resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(112, 112, 3))
    model.add(TimeDistributed(resnet, input_shape=input_shape))
    model.add(TimeDistributed(Flatten()))

    # LSTM for sequence analysis
    model.add(LSTM(128, return_sequences=True))
    model.add(Dropout(0.5))
    model.add(LSTM(64, return_sequences=False))
    model.add(Dropout(0.5))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
    return model





In [ ]:
# Function to plot training history
def plot_history(history):
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    plt.savefig('training_history.png')
    plt.close()


In [ ]:
def plot_confusion_matrix(y_true, y_pred, save_path):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title('Confusion Matrix')
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.savefig(save_path)
    plt.close()
    print(f"Confusion matrix saved to {save_path}")

In [ ]:
import os
import numpy as np
import zipfile

def load_and_preprocess_dataset(real_dir, fake_dir, preprocessed_data_path):
    try:
        if os.path.exists(preprocessed_data_path):
            print("Loading preprocessed dataset...")
            with np.load(preprocessed_data_path) as data:
                X = data['X']
                y = data['y']
        else:
            raise FileNotFoundError("Preprocessed data file not found.")
    except (FileNotFoundError, zipfile.BadZipFile):
        print("Preprocessed data not found or corrupted. Processing raw dataset...")
        X, y = load_dataset(real_dir, fake_dir)
        np.savez_compressed(preprocessed_data_path, X=X, y=y)

    return X, y

In [ ]:
def train_model(X, y):
    # Split the dataset
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Build and train the model
    input_shape = (10, 112, 112, 3)  # (num_frames, height, width, channels)
    model = build_model(input_shape)

    # Define callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    model_checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_accuracy')

    # Train the model
    history = model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=100,
        batch_size=16,
        callbacks=[early_stopping, model_checkpoint]
    )

    # Plot and save training history
    plot_history(history)

    # Evaluate the model
    y_pred = model.predict(X_test)
    y_pred_classes = (y_pred > 0.5).astype(int).flatten()

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred_classes)
    precision = precision_score(y_test, y_pred_classes)
    recall = recall_score(y_test, y_pred_classes)
    f1 = f1_score(y_test, y_pred_classes)

    print(f"Test Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")

     # Plot and save confusion matrix
    plot_confusion_matrix(y_test, y_pred_classes, 'confusion_matrix.png')

    return model, accuracy, precision, recall, f1


In [ ]:
def predict_video(video_path, model):
    frames = extract_frames(video_path)
    frames = np.expand_dims(frames, axis=0)  # Add batch dimension
    prediction = model.predict(frames)
    return prediction[0][0]

In [ ]:
if __name__ == "__main__":
    real_dir = "/content/drive/MyDrive/RFvideos/real"
    fake_dir = "/content/drive/MyDrive/RFvideos/fake"
    preprocessed_data_path = "/content/drive/MyDrive/RFvideos/preprocessed_data.npz"
    model_path = "/content/drive/MyDrive/RFvideos/best_model.keras"

    X, y = load_and_preprocess_dataset(real_dir, fake_dir, preprocessed_data_path)

    if os.path.exists(model_path):
        print("Loading existing model...")
        model = load_model(model_path)

        # Evaluate the model
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        y_pred = model.predict(X_test)
        y_pred_classes = (y_pred > 0.5).astype(int).flatten()

        # Calculate metrics
        accuracy = accuracy_score(y_test, y_pred_classes)
        precision = precision_score(y_test, y_pred_classes)
        recall = recall_score(y_test, y_pred_classes)
        f1 = f1_score(y_test, y_pred_classes)

        print(f"Test Accuracy: {accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1 Score: {f1:.4f}")

        # Plot and save confusion matrix
        plot_confusion_matrix(y_test, y_pred_classes, 'confusion_matrix.png')

    else:
        print("Training new model...")
        model, accuracy, precision, recall, f1 = train_model(X, y)

   # Test the model on a new video
    test_video_path = "/content/drive/MyDrive/RFvideos/01__kitchen_pan.mp4"
    result = predict_video(test_video_path, model)
    print(f"Prediction: {result:.4f}")
    print(f"The video is {'likely fake' if result > 0.5 else 'likely real'}")



Loading preprocessed dataset...
Loading existing model...
2/2 ━━━━━━━━━━━━━━━━━━━━ 66s 25s/step
Test Accuracy: 0.8750
Precision: 0.8500
Recall: 0.8947
F1 Score: 0.8718
Confusion matrix saved to confusion_matrix.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 611ms/step
Prediction: 0.3117
The video is likely real
